In [1]:
import zipfile
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
import pandas as pd
import os
import numpy as np
from os import listdir
from slugify import slugify
import requests
from  urllib3.exceptions import ReadTimeoutError
from urllib.error import HTTPError
from urllib.parse import urlparse
from PIL import Image, UnidentifiedImageError
import warnings
warnings.filterwarnings('ignore')
from google.colab import files
import spacy
nlp=spacy.load('en_core_web_sm')

In [2]:
!unzip -u /content/drive/MyDrive/all_data.zip

Archive:  /content/drive/MyDrive/all_data.zip
  inflating: all_data.csv            


In [5]:
news_data=pd.read_csv('all_data.csv',index_col=[0,1]).reset_index(drop=True)

In [6]:
news_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20015 entries, 0 to 20014
Data columns (total 52 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              20015 non-null  object 
 1   comments            16844 non-null  float64
 2   country             20015 non-null  object 
 3   crawled             16844 non-null  object 
 4   domain_rank         20015 non-null  int64  
 5   id                  4903 non-null   float64
 6   language            16844 non-null  object 
 7   likes               16844 non-null  float64
 8   main_img_url        20015 non-null  object 
 9   ord_in_thread       16844 non-null  float64
 10  participants_count  16844 non-null  float64
 11  published           13514 non-null  object 
 12  replies_count       16844 non-null  float64
 13  shares              16844 non-null  float64
 14  site_url            16844 non-null  object 
 15  spam_score          16844 non-null  float64
 16  text

In [7]:
news_data.head()

,author,comments,country,crawled,domain_rank,id,language,likes,main_img_url,ord_in_thread,...,fear,joy,sadness,surprise,trust,negative,positive,first_all,second_all,third_all
0,JEREMY W. PETERS,0.0,US,2017-03-14 08:25:04,0,3.0,english,0.0,https://static01.nyt.com/images/2016/11/23/us/...,0.0,...,6,20,5,14,30,14,52,409,150,45
1,STEVE EDER,0.0,US,2017-03-14 08:25:36,0,4.0,english,0.0,https://static01.nyt.com/images/2016/10/18/us/...,0.0,...,4,4,4,5,9,8,15,145,38,28
2,MAGGIE HABERMAN ASHLEY PARKER,0.0,US,2017-03-14 08:25:36,0,5.0,english,0.0,https://static01.nyt.com/images/2016/11/12/us/...,0.0,...,8,15,8,6,26,15,34,440,168,68
3,NELSON D. SCHWARTZ SUI-LEE WEE,0.0,US,2017-03-14 08:25:36,0,6.0,english,0.0,https://static01.nyt.com/images/2016/10/21/bus...,0.0,...,10,10,10,6,32,24,43,592,136,71
4,MAGGIE HABERMAN,0.0,US,2017-03-14 08:25:37,0,7.0,english,0.0,https://static01.nyt.com/images/2016/11/21/us/...,0.0,...,3,6,2,4,14,4,25,223,82,28


In [8]:
news_data["language"].value_counts()

english       16252
russian         203
spanish         172
german          111
french           37
arabic           22
portuguese       11
turkish          10
italian           9
greek             4
ignore            4
dutch             3
norwegian         3
finnish           1
chinese           1
polish            1
Name: language, dtype: int64

In [9]:
news_data=news_data[news_data['language']=='english']

In [10]:
news_data["language"].value_counts()

english    16252
Name: language, dtype: int64

In [11]:
news_data=news_data[["type","text","title","main_img_url","site_url"]].reset_index(drop=True)

In [13]:
news_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16252 entries, 0 to 16251
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          16252 non-null  object
 1   text          16252 non-null  object
 2   title         16252 non-null  object
 3   main_img_url  16252 non-null  object
 4   site_url      16252 non-null  object
dtypes: object(5)
memory usage: 635.0+ KB


In [14]:
from requests.exceptions import ConnectionError
def check_url_exists(url: str):
    """
    Checks if a url exists
    :param url: url to check
    :return: True if the url exists, false otherwise.
    """

    # print(url)
    return requests.head(url, allow_redirects=True).status_code == 200

In [15]:
def is_url_image(image_url):
   image_formats = ("image/png", "image/jpeg", "image/jpg")
   r = requests.head(image_url)
   if r.headers.get("content-type",'') in image_formats:
      return True
   return False

In [16]:
def uri_validator(x):
    try:
        result = urlparse(x)
        return all([result.scheme, result.netloc])
    except:
        return False


In [17]:
def saving_img(img_url,i):
   ext=str(img_url).split(".")[-1]
   format = 'JPEG' if ext.lower() == 'jpg' else ext.upper()
   response = requests.get(img_url,timeout=5)
   if response.status_code:
       img = Image.open(BytesIO(response.content))
       if img.mode in ("RGBA", "P"):
        img = img.convert("RGB")
       img.save("images_news_all/"+str(i)+f"{slugify(img_url[0:100])}.{ext}",format)
       print("saving",i)

In [ ]:
## Before running this block, create a new folder where all retrieved images will be saved
from requests.exceptions import SSLError
delete_index=[]
for i,url in enumerate(news_data["main_img_url"]):
     try:
       res=is_url_image(url)
       if(res==False):
          delete_index.append(i)
          continue
       response = requests.head(url, allow_redirects=True)
       saving_img(url,i)
     except (SSLError,KeyError,UnidentifiedImageError, ConnectionError,requests.exceptions.MissingSchema, ReadTimeoutError,requests.exceptions.Timeout):
       delete_index.append(i)
       continue

In [ ]:
news_data.drop(delete_index, inplace=True)
news_data.reset_index(drop=True,inplace=True)

In [ ]:
dir_path="/content/images_news_all"
count = 0
ima=[]
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
       ima.append(path)
print('File count:', len(ima))

In [ ]:
index=[]
images=[]
for i,path in enumerate(ima):
   ind=ima[i].split('h')[0]
   images.append(path)
   index.append(ind)

In [ ]:
for i,path in enumerate(ima):
   ind=ima[i].split('h')[0]
   for img_url in news_data['main_img_url']:
       if ind in img_url:
          news_data["img"]=ima

In [ ]:
index=sorted(index,key=int)
news_data=news_data.iloc[index]